In [5]:
# %load tensor.py
# 딥러닝 모델 관련 모듈
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import numpy as np
import pandas as pd

In [8]:
# mnist 데이터셋 로드
(X_train,y_train),(X_test,y_test) = mnist.load_data()

In [9]:
# CNN을 실행하기 위해 데이터 reshape
X_train = X_train.reshape(X_train.shape[0],28,28,1).astype('float32')/255
X_test = X_test.reshape(X_test.shape[0],28,28,1).astype('float32')/255

y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

In [ ]:
# CNN 신경망 설정
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(28,28,1),activation='relu')) # 커널 갯수 32 사이즈 3*3
model.add(Conv2D(64,(3,3),activation='relu')) # 커널 64개
model.add(MaxPooling2D(pool_size=(2,2))) # 풀링 사이즈(2,2) -> 50% 줄어듬
model.add(Dropout(0.25)) # 노드 25% 꺼줌(계산 안함)
# Dense 실행 -> 1차원으로 데이터타입을 변경
model.add(Flatten()) # (28,28,1) -> (28*28*1) 
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 최적화를 위한 설정
model_path= './data/model/MNIST_MLP.hdf5'
checkpointer = ModelCheckpoint(filepath=model_path, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

hist = model.fit(X_train, y_train, epochs=2000,
                 validation_split=0.25, batch_size=200, verbose=0,
                 callbacks=[early_stopping, checkpointer])

# 테스트 정확도 출력
print("Test accuracy : ", model.evaluate(X_test, y_test)[1])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_9 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 12, 12, 64)        0         
                                                                 
 flatten_4 (Flatten)         (None, 9216)              0         
                                                                 
 dense_8 (Dense)             (None, 128)               1179776   
                                                      